In [221]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from helpers import get_abspath


In [254]:
# get datasets for error and accuracy curves
resdir = 'results/NN'
BP = pd.read_csv(get_abspath('BP/results.csv', resdir))
RHC = pd.read_csv(get_abspath('RHC/results.csv', resdir))
SA = pd.read_csv(get_abspath('SA/results_10000000000.0_0.15.csv', resdir))
GA = pd.read_csv(get_abspath('GA/results_100_10_10.csv', resdir))

# rename columns
bCols = {'MSE_train': 'bp_msetrain', 'MSE_test': 'bp_msetest', 'MSE_validation': 'bp_msevalid', 'acc_train': 'bp_acctrain', 'acc_test': 'bp_acctest', 'acc_validation': 'bp_accvalid', 'seconds_elapsed': 'bp_time'}
rCols = {'MSE_train': 'rhc_msetrain', 'MSE_test': 'rhc_msetest', 'MSE_validation': 'rhc_msevalid', 'acc_train': 'rhc_acctrain', 'acc_test': 'rhc_acctest', 'acc_validation': 'rhc_accvalid', 'seconds_elapsed': 'rhc_time'}
sCols = {'MSE_train': 'sa_msetrain', 'MSE_test': 'sa_msetest', 'MSE_validation': 'sa_msevalid', 'acc_train': 'sa_acctrain', 'acc_test': 'sa_acctest', 'acc_validation': 'sa_accvalid', 'seconds_elapsed': 'sa_time'}
gCols = {'MSE_train': 'ga_msetrain', 'MSE_test': 'ga_msetest', 'MSE_validation': 'ga_msevalid', 'acc_train': 'ga_acctrain', 'acc_test': 'ga_acctest', 'acc_validation': 'ga_accvalid', 'seconds_elapsed': 'ga_time'}

BP = df['BP'].rename(index=str, columns=bpCols)
RHC = df['RHC'].drop(columns='iteration').rename(index=str, columns=rCols)
SA = df['SA'].drop(columns='iteration').rename(index=str, columns=sCols)
GA = df['GA'].drop(columns='iteration').rename(index=str, columns=gCols)

KeyError: 'BP'

In [250]:
(BP.pipe(rename))

NameError: name 'rename' is not defined

In [253]:
BP %>% rename(index=str, columns='iteration')

SyntaxError: invalid syntax (<ipython-input-253-4dbca5e957b6>, line 1)